# First of all let's see what we will do
- Frame the dataset and check the info()
    * here we can notice that the is no null data, as there are no missing values, but let's not get tricked with this bait.
- For string values, we need to do the value_count(), we can check the anomalies in names
    * The gender column is Ok, has no weird values.
    * The Partner is Ok, has only values of Yes and No. No upnormal values detected.
    * The Dependents Column is Ok, only values of Yes and No. No upnormal values detected.
    * Phone Service is normal, has no anomalies.
    * MutipleLines, we have noticed an anomaly, that some customers don't have any internet phone service at all.
        * For this one we will check if the customers who don't have phone service, have any other services or not, if not we may drop them
        * We have noticed that not having a phone service doesn't mean that we don't have internet service, so we will just change them to Yes and No, as we have noticed that the DSL owners, might have mutiple lines. BUT, we can notice that all the people who don't have phone service, do have a DSL internet.
    * InternetService, has no anomalies. But the type of internet service we will just set it to Yes or No, as the type of the connection won't make difference
    * now from OnlineSecurity to StreamingMovies, we have no anomalies, but we still have the same thing as the MultipleLines but with InternetService, as if someone who don't have internet service won't be able to do these things. So, we will just change the values to Yes or No.
    * Contact has no anomalies, but we will change the type to an integer, showing the number of months for the contract.
    * PaperLess Billing has no anomalies.
    * PaymentMethod is normal, has no anomalies, but we can change it to be EPayment, and just yes or no values, as I think, knowing the type of payment won't make any difference but maybe knowing if it's electronic or not will make.

- For numerical values, we need to do the describe(), we can check the outliers in the data
    * Senior Citizen, has no outliers, and no upnormal values.
    * Tenure is normal, no anomalies detected.
    * MonthlyCharges are normal, there are no extreme outliers.
    * TotalCharges column needs to be casted to float datatype, for that, we will first need to fill the empty values as they can't be casted directly, we are going to fill them with zero => cast them to float, and calculate their values in case the tenure wasn't 0

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df.sample(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
526,8606-CIQUL,Male,1,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),115.75,8399.15,No
939,0347-UBKUZ,Female,0,No,No,15,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.90,320.45,No
1257,0723-VSOBE,Female,1,No,No,45,No,No phone service,DSL,No,No,No,No,No,Yes,One year,No,Electronic check,34.20,1596.6,No
6591,5832-XKAES,Male,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),69.80,134.7,Yes
182,9367-WXLCH,Male,0,No,No,8,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,No,Bank transfer (automatic),84.50,662.65,Yes
5555,8750-QWZAJ,Female,0,Yes,Yes,70,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.80,1378.75,No
1026,4822-RVYBB,Male,1,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,100.60,819.4,Yes
1995,8000-REIQB,Female,1,No,No,1,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,69.95,69.95,Yes
2924,1428-GTBJJ,Male,0,No,No,11,Yes,No,Fiber optic,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,74.55,824.75,Yes
4410,0848-ZGQIJ,Female,0,Yes,No,16,Yes,No,Fiber optic,No,Yes,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),94.65,1461.15,No


In [6]:
df['gender'].value_counts()

gender
Male      3555
Female    3488
Name: count, dtype: int64

In [7]:
df['SeniorCitizen'].describe()

count    7043.000000
mean        0.162147
std         0.368612
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: SeniorCitizen, dtype: float64

In [8]:
df['SeniorCitizen'].value_counts()

SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64

In [9]:
df['Partner'].value_counts()

Partner
No     3641
Yes    3402
Name: count, dtype: int64

In [10]:
df["Dependents"].value_counts()

Dependents
No     4933
Yes    2110
Name: count, dtype: int64

In [11]:
df["tenure"].describe()

count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

In [12]:
df["PhoneService"].value_counts()

PhoneService
Yes    6361
No      682
Name: count, dtype: int64

In [13]:
df["MultipleLines"].value_counts()

MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64

In [14]:
test_df = df[df["MultipleLines"] == "No phone service"]

In [15]:
test_df["InternetService"].value_counts()

InternetService
DSL    682
Name: count, dtype: int64

In [16]:
def fix_MutipleLines_issue(value):
    if "No" in value:
        return "No"
    else:
        return "Yes"

In [17]:
df["MultipleLines"] = df["MultipleLines"].apply(fix_MutipleLines_issue)

In [18]:
df.sample(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6132,3896-RCYYE,Female,0,No,No,67,No,No,DSL,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),53.40,3579.15,No
6748,6230-BSUXY,Female,1,No,No,1,Yes,No,Fiber optic,No,No,Yes,No,No,Yes,Month-to-month,Yes,Electronic check,85.00,85,Yes
216,9286-BHDQG,Male,0,Yes,Yes,72,No,No,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Credit card (automatic),45.25,3139.8,No
1250,6766-HFKLA,Female,0,Yes,No,56,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Two year,No,Bank transfer (automatic),103.20,5744.35,No
2550,8129-GMVGI,Female,0,Yes,Yes,56,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.05,1090.1,No
3699,1137-DGOWI,Female,0,Yes,No,30,Yes,No,DSL,No,Yes,Yes,Yes,No,Yes,One year,No,Bank transfer (automatic),70.25,2198.9,No
3053,0960-HUWBM,Male,0,Yes,Yes,65,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),104.10,6700.05,No
1333,0121-SNYRK,Male,0,No,No,50,No,No,DSL,Yes,No,No,Yes,No,No,One year,Yes,Mailed check,35.40,1748.9,No
2631,6861-XWTWQ,Male,1,Yes,No,7,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.25,665.45,Yes
4429,3486-HOOGQ,Female,0,Yes,Yes,70,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Bank transfer (automatic),79.70,5743.3,No


In [19]:
df["InternetService"].value_counts()

InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64

In [20]:
df["OnlineSecurity"].value_counts()

OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: count, dtype: int64

In [21]:
df["OnlineBackup"].value_counts()

OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: count, dtype: int64

In [22]:
df["DeviceProtection"].value_counts()

DeviceProtection
No                     3095
Yes                    2422
No internet service    1526
Name: count, dtype: int64

In [23]:
df["TechSupport"].value_counts()

TechSupport
No                     3473
Yes                    2044
No internet service    1526
Name: count, dtype: int64

In [24]:
df["StreamingTV"].value_counts()

StreamingTV
No                     2810
Yes                    2707
No internet service    1526
Name: count, dtype: int64

In [25]:
df["StreamingMovies"].value_counts()

StreamingMovies
No                     2785
Yes                    2732
No internet service    1526
Name: count, dtype: int64

In [26]:
df["StreamingMovies"].value_counts()

StreamingMovies
No                     2785
Yes                    2732
No internet service    1526
Name: count, dtype: int64

In [27]:
def fix_no_internet_service(value):
    if "No" in value:
        return "No"
    else:
        return "Yes"

In [28]:
df.sample(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4460,0365-GXEZS,Male,0,Yes,No,18,Yes,No,Fiber optic,No,No,Yes,Yes,No,No,Month-to-month,No,Electronic check,78.20,1468.75,No
855,2382-BCKQJ,Female,0,No,Yes,62,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,One year,Yes,Electronic check,100.15,6413.65,Yes
7016,1471-GIQKQ,Female,0,No,No,1,Yes,No,DSL,No,Yes,No,No,No,No,Month-to-month,No,Electronic check,49.95,49.95,No
5307,1941-HOSAM,Male,0,Yes,Yes,1,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,90.10,90.1,No
680,1448-PWKYE,Male,0,Yes,Yes,1,Yes,No,Fiber optic,No,No,No,No,No,Yes,Month-to-month,No,Electronic check,80.00,80,Yes
5730,7167-PCEYD,Male,0,No,No,67,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,19.80,1311.3,No
6471,0859-YGKFW,Male,0,Yes,Yes,18,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Bank transfer (automatic),20.05,345.9,No
2891,0495-RVCBF,Female,0,No,No,1,Yes,No,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,79.70,79.7,Yes
5676,9103-CXVOK,Male,0,Yes,Yes,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,19.75,19.75,No
4609,2519-TWKFS,Male,0,Yes,Yes,25,Yes,Yes,DSL,No,No,No,Yes,No,No,One year,Yes,Mailed check,55.10,1466.1,No


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [30]:
df['OnlineSecurity'] = df['OnlineSecurity'].apply(fix_no_internet_service)

In [31]:
df['OnlineBackup'] = df['OnlineBackup'].apply(fix_no_internet_service)

In [32]:
df['DeviceProtection'] = df['DeviceProtection'].apply(fix_no_internet_service)

In [33]:
df['TechSupport'] = df['TechSupport'].apply(fix_no_internet_service)

In [34]:
df['StreamingTV'] = df['StreamingTV'].apply(fix_no_internet_service)

In [35]:
df['StreamingMovies'] = df['StreamingMovies'].apply(fix_no_internet_service)

In [36]:
df.sample(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3048,1728-BQDMA,Female,0,No,No,2,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,No,Electronic check,44.45,82.7,No
2092,2251-PYLPB,Male,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,No,Yes,Two year,No,Credit card (automatic),79.55,5810.9,No
1813,0817-HSUSE,Male,0,No,No,1,Yes,No,DSL,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,75.50,75.5,No
5987,1099-GODLO,Female,0,Yes,Yes,1,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.35,20.35,No
3919,9025-AOMKI,Female,0,No,No,6,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),93.55,536.4,Yes
1273,9681-KYGYB,Male,1,Yes,No,49,Yes,Yes,Fiber optic,Yes,Yes,Yes,No,No,No,Month-to-month,Yes,Electronic check,88.20,4159.45,No
699,8338-QIUNR,Male,0,Yes,Yes,72,Yes,Yes,DSL,Yes,No,Yes,Yes,No,No,Two year,Yes,Credit card (automatic),66.50,4811.6,No
3810,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,No
4502,1264-BYWMS,Male,0,No,No,55,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,96.80,5283.95,Yes
6898,1956-YIFGE,Male,0,Yes,Yes,22,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Mailed check,100.05,2090.25,No


In [37]:
df.sample(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1146,1430-SFQSA,Male,0,No,No,29,No,No,DSL,Yes,No,No,Yes,No,No,One year,No,Mailed check,35.60,1072.6,No
5395,9095-HFAFX,Female,0,No,No,5,Yes,No,Fiber optic,No,No,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),81.00,389.6,Yes
1982,3190-FZATL,Male,0,No,Yes,14,Yes,No,No,No,No,No,No,No,No,Month-to-month,No,Mailed check,19.95,268.4,No
1607,4154-AQUGT,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,No,Yes,No,Yes,Month-to-month,Yes,Bank transfer (automatic),89.05,1169.35,Yes
5970,4853-OITSN,Male,0,Yes,No,5,Yes,Yes,Fiber optic,Yes,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,89.15,413.25,No
210,7841-TZDMQ,Male,0,No,No,2,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),46.00,84.5,Yes
1258,5529-GIBVH,Female,0,No,No,47,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.15,1046.2,No
1041,1915-OAKWD,Female,0,No,Yes,16,Yes,No,No,No,No,No,No,No,No,One year,Yes,Bank transfer (automatic),19.30,360.35,No
736,6647-ZEDXT,Female,0,No,No,30,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,One year,No,Credit card (automatic),110.45,3327.05,No
5457,9496-IVVRP,Female,0,Yes,Yes,11,Yes,No,DSL,Yes,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),51.00,581.7,No


In [38]:
df['Contract'].value_counts()
# Keep in mind, you might want to change them to a numeric values that express the number of months, month-to-month => 1, Two year => 24, One year => 12
# so that we might try to match the relation with the tenure

Contract
Month-to-month    3875
Two year          1695
One year          1473
Name: count, dtype: int64

In [39]:
df['PaperlessBilling'].value_counts()

PaperlessBilling
Yes    4171
No     2872
Name: count, dtype: int64

In [40]:
df['PaymentMethod'].value_counts()

PaymentMethod
Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: count, dtype: int64

In [41]:
df['MonthlyCharges'].describe()

count    7043.000000
mean       64.761692
std        30.090047
min        18.250000
25%        35.500000
50%        70.350000
75%        89.850000
max       118.750000
Name: MonthlyCharges, dtype: float64

In [42]:
df['TotalCharges'].describe()

count     7043
unique    6531
top       20.2
freq        11
Name: TotalCharges, dtype: object

In [43]:
df['TotalCharges']

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object

In [44]:
display(df.loc[df['TotalCharges'] == " "])

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No,No,No,No,No,No,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No,No,No,No,No,No,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [45]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [46]:
def fill_empty_total_charges(value):
    if value == " ":
        return "0"
    else:
        return value

In [47]:
df['TotalCharges'] = df['TotalCharges'].apply(fill_empty_total_charges)

In [48]:
df['TotalCharges'] = df['TotalCharges'].astype(float)

In [49]:
df['TotalCharges'] = df['TotalCharges'].round(2)

In [50]:
df.round(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [51]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [52]:
df['PaperlessBilling'].value_counts()

PaperlessBilling
Yes    4171
No     2872
Name: count, dtype: int64

In [53]:
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,No
7,6713-OKOMC,Female,0,No,No,10,No,No,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.90,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [54]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [55]:
df['Churn'].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

# New Phase feature Engineering phase
- First we will create a new column that in case this column will contain if the payment is manual then 1 else 0, and we will check the correlation with the churn colum, and at the same time we will change the churn column to be a 1 for yes and 0 for no
    * We found that the correlation here wasn't enough, so we will try another thing in the third step.
- Second, we are going to change the Churn to be 1 for yes and 0 for no
- Third, we are going to see if the payment is electronic or pyhsical, if electronic then 1 else 0

In [56]:
df.groupby('PaymentMethod')['Churn'].value_counts('Yes')

PaymentMethod              Churn
Bank transfer (automatic)  No       0.832902
                           Yes      0.167098
Credit card (automatic)    No       0.847569
                           Yes      0.152431
Electronic check           No       0.547146
                           Yes      0.452854
Mailed check               No       0.808933
                           Yes      0.191067
Name: proportion, dtype: float64

In [57]:
df.groupby('PaymentMethod')['Churn'].agg(lambda x: x.mode()[0])

PaymentMethod
Bank transfer (automatic)    No
Credit card (automatic)      No
Electronic check             No
Mailed check                 No
Name: Churn, dtype: object

In [58]:
# Now we will change the Churn, first starting by a function to change the values:
def change_churn(value):
    if 'Yes' in value:
        return 1
    else:
        return 0

In [59]:
df['Churn'] = df['Churn'].apply(change_churn)

In [60]:
df['Churn'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7043 entries, 0 to 7042
Series name: Churn
Non-Null Count  Dtype
--------------  -----
7043 non-null   int64
dtypes: int64(1)
memory usage: 55.2 KB


In [61]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [62]:
# Now let's split the different types of internet into different columns
df['InternetService'].value_counts()

InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64

In [63]:
df['DSL_connection'] = df['InternetService'].apply(lambda x : x == 'DSL')

In [64]:
df['Fiber_connection'] = df['InternetService'].apply(lambda x : x == 'Fiber optic')

In [65]:
# For this one, in case the customer has internet it will return true, else, it will return false.
df['has_internet'] = df['InternetService'].apply(lambda x : x != 'No')

In [66]:
# Now for the contract type, we are going to make three new columns, 1- monthly contract, 2- annual contract, 3- two_years_contract
df['montly_contract'] = df['Contract'].str.contains('Month')

In [67]:
df['annual_contract'] = df['Contract'].str.contains('One year')

In [68]:
df['Contract'].value_counts()

Contract
Month-to-month    3875
Two year          1695
One year          1473
Name: count, dtype: int64

In [69]:
df['two_years_contract'] = df['Contract'].str.contains('Two year')

In [70]:
# Now the customerID Column has no effect, so, I'm going to drop it
df.drop('customerID', axis=1, inplace=True)

In [71]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,DSL_connection,Fiber_connection,has_internet,montly_contract,annual_contract,two_years_contract
0,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,True,False,True,True,False,False
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0,True,False,True,False,True,False
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,True,False,True,True,False,False
3,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,True,False,True,False,True,False
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,False,True,True,True,False,False


In [72]:
# Now let's make different columns for different payment method.
df['electronic_check_payment'] = df['PaymentMethod'].str.contains('Electronic check')
df['mailed_check_payment'] = df['PaymentMethod'].str.contains('Mailed check')
df['bank_transfer_payment'] = df['PaymentMethod'].str.contains('Bank transfer')
df['credit_card_payment'] = df['PaymentMethod'].str.contains('Credit card')

In [73]:
# Now we want to combine the streamingTV and streamingMovies into one column called => streamsTV
# This logic is going to be made based on the fact the either one of the two is true
df['streamsTV'] = (df['StreamingMovies'] == 'Yes') | (df['StreamingTV'] == 'Yes')

In [74]:
# Now let's drop the InternetService, StreamingTV, StreamingMovies, contract, and PaymentMethod columns as we encoded them into different columns
df.drop('InternetService', axis=1, inplace=True)
df.drop('StreamingTV', axis=1, inplace=True)
df.drop('StreamingMovies', axis=1, inplace=True)
df.drop('Contract', axis=1, inplace=True)
df.drop('PaymentMethod', axis=1, inplace=True)

In [75]:
df.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,DSL_connection,Fiber_connection,has_internet,montly_contract,annual_contract,two_years_contract,electronic_check_payment,mailed_check_payment,bank_transfer_payment,credit_card_payment,streamsTV
0,Female,0,Yes,No,1,No,No,No,Yes,No,No,Yes,29.85,29.85,0,True,False,True,True,False,False,True,False,False,False,False
1,Male,0,No,No,34,Yes,No,Yes,No,Yes,No,No,56.95,1889.50,0,True,False,True,False,True,False,False,True,False,False,False
2,Male,0,No,No,2,Yes,No,Yes,Yes,No,No,Yes,53.85,108.15,1,True,False,True,True,False,False,False,True,False,False,False
3,Male,0,No,No,45,No,No,Yes,No,Yes,Yes,No,42.30,1840.75,0,True,False,True,False,True,False,False,False,True,False,False
4,Female,0,No,No,2,Yes,No,No,No,No,No,Yes,70.70,151.65,1,False,True,True,True,False,False,True,False,False,False,False
5,Female,0,No,No,8,Yes,Yes,No,No,Yes,No,Yes,99.65,820.50,1,False,True,True,True,False,False,True,False,False,False,True
6,Male,0,No,Yes,22,Yes,Yes,No,Yes,No,No,Yes,89.10,1949.40,0,False,True,True,True,False,False,False,False,False,True,True
7,Female,0,No,No,10,No,No,Yes,No,No,No,No,29.75,301.90,0,True,False,True,True,False,False,False,True,False,False,False
8,Female,0,Yes,No,28,Yes,Yes,No,No,Yes,Yes,Yes,104.80,3046.05,1,False,True,True,True,False,False,True,False,False,False,True
9,Male,0,No,Yes,62,Yes,No,Yes,Yes,No,No,No,56.15,3487.95,0,True,False,True,False,True,False,False,False,True,False,False


In [76]:
# Now for the gender we are going to make the one hot encoding using Sklearn
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
one_hot_encoded = encoder.fit_transform(df[['gender']])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['gender']))
df = pd.concat([df.drop(['gender'], axis=1), one_hot_df], axis=1)
df
# continue adding this one_hot_df for the data frame
# also split the data and ensure you remove the outliers.

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,DSL_connection,Fiber_connection,has_internet,montly_contract,annual_contract,two_years_contract,electronic_check_payment,mailed_check_payment,bank_transfer_payment,credit_card_payment,streamsTV,gender_Female,gender_Male
0,0,Yes,No,1,No,No,No,Yes,No,No,Yes,29.85,29.85,0,True,False,True,True,False,False,True,False,False,False,False,1.0,0.0
1,0,No,No,34,Yes,No,Yes,No,Yes,No,No,56.95,1889.50,0,True,False,True,False,True,False,False,True,False,False,False,0.0,1.0
2,0,No,No,2,Yes,No,Yes,Yes,No,No,Yes,53.85,108.15,1,True,False,True,True,False,False,False,True,False,False,False,0.0,1.0
3,0,No,No,45,No,No,Yes,No,Yes,Yes,No,42.30,1840.75,0,True,False,True,False,True,False,False,False,True,False,False,0.0,1.0
4,0,No,No,2,Yes,No,No,No,No,No,Yes,70.70,151.65,1,False,True,True,True,False,False,True,False,False,False,False,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,Yes,Yes,24,Yes,Yes,Yes,No,Yes,Yes,Yes,84.80,1990.50,0,True,False,True,False,True,False,False,True,False,False,True,0.0,1.0
7039,0,Yes,Yes,72,Yes,Yes,No,Yes,Yes,No,Yes,103.20,7362.90,0,False,True,True,False,True,False,False,False,False,True,True,1.0,0.0
7040,0,Yes,Yes,11,No,No,Yes,No,No,No,Yes,29.60,346.45,0,True,False,True,True,False,False,True,False,False,False,False,1.0,0.0
7041,1,Yes,No,4,Yes,Yes,No,No,No,No,Yes,74.40,306.60,1,False,True,True,True,False,False,False,True,False,False,False,0.0,1.0


### Now we are going to start working on changing the Yes, No columns to 1 and 0.
- As the OneHotEncoding isn't the best to do this as it's not very memory efficient in this case, and also we don't want to split to several column.

In [77]:
yes_no_transformation_columns = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'PaperlessBilling']
for column in yes_no_transformation_columns:
    df[column] = df[column].map({'Yes': 1, 'No':0})

df.head(10)

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,DSL_connection,Fiber_connection,has_internet,montly_contract,annual_contract,two_years_contract,electronic_check_payment,mailed_check_payment,bank_transfer_payment,credit_card_payment,streamsTV,gender_Female,gender_Male
0,0,1,0,1,0,0,0,1,0,0,1,29.85,29.85,0,True,False,True,True,False,False,True,False,False,False,False,1.0,0.0
1,0,0,0,34,1,0,1,0,1,0,0,56.95,1889.50,0,True,False,True,False,True,False,False,True,False,False,False,0.0,1.0
2,0,0,0,2,1,0,1,1,0,0,1,53.85,108.15,1,True,False,True,True,False,False,False,True,False,False,False,0.0,1.0
3,0,0,0,45,0,0,1,0,1,1,0,42.30,1840.75,0,True,False,True,False,True,False,False,False,True,False,False,0.0,1.0
4,0,0,0,2,1,0,0,0,0,0,1,70.70,151.65,1,False,True,True,True,False,False,True,False,False,False,False,1.0,0.0
5,0,0,0,8,1,1,0,0,1,0,1,99.65,820.50,1,False,True,True,True,False,False,True,False,False,False,True,1.0,0.0
6,0,0,1,22,1,1,0,1,0,0,1,89.10,1949.40,0,False,True,True,True,False,False,False,False,False,True,True,0.0,1.0
7,0,0,0,10,0,0,1,0,0,0,0,29.75,301.90,0,True,False,True,True,False,False,False,True,False,False,False,1.0,0.0
8,0,1,0,28,1,1,0,0,1,1,1,104.80,3046.05,1,False,True,True,True,False,False,True,False,False,False,True,1.0,0.0
9,0,0,1,62,1,0,1,1,0,0,0,56.15,3487.95,0,True,False,True,False,True,False,False,False,True,False,False,0.0,1.0


In [78]:
# Ok Now lets loop over the True or False columns and map them to {'True' : 1, 'False': 0}
true_false_transformation_columns = ['DSL_connection', 'Fiber_connection', 'has_internet', 'montly_contract', 'annual_contract', 'two_years_contract', 'electronic_check_payment', 'mailed_check_payment', 'bank_transfer_payment', 'credit_card_payment', 'streamsTV']
for column in true_false_transformation_columns:
    df[column] = df[column].map({True : 1, False: 0})

df.head(10)

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,DSL_connection,Fiber_connection,has_internet,montly_contract,annual_contract,two_years_contract,electronic_check_payment,mailed_check_payment,bank_transfer_payment,credit_card_payment,streamsTV,gender_Female,gender_Male
0,0,1,0,1,0,0,0,1,0,0,1,29.85,29.85,0,1,0,1,1,0,0,1,0,0,0,0,1.0,0.0
1,0,0,0,34,1,0,1,0,1,0,0,56.95,1889.50,0,1,0,1,0,1,0,0,1,0,0,0,0.0,1.0
2,0,0,0,2,1,0,1,1,0,0,1,53.85,108.15,1,1,0,1,1,0,0,0,1,0,0,0,0.0,1.0
3,0,0,0,45,0,0,1,0,1,1,0,42.30,1840.75,0,1,0,1,0,1,0,0,0,1,0,0,0.0,1.0
4,0,0,0,2,1,0,0,0,0,0,1,70.70,151.65,1,0,1,1,1,0,0,1,0,0,0,0,1.0,0.0
5,0,0,0,8,1,1,0,0,1,0,1,99.65,820.50,1,0,1,1,1,0,0,1,0,0,0,1,1.0,0.0
6,0,0,1,22,1,1,0,1,0,0,1,89.10,1949.40,0,0,1,1,1,0,0,0,0,0,1,1,0.0,1.0
7,0,0,0,10,0,0,1,0,0,0,0,29.75,301.90,0,1,0,1,1,0,0,0,1,0,0,0,1.0,0.0
8,0,1,0,28,1,1,0,0,1,1,1,104.80,3046.05,1,0,1,1,1,0,0,1,0,0,0,1,1.0,0.0
9,0,0,1,62,1,0,1,1,0,0,0,56.15,3487.95,0,1,0,1,0,1,0,0,0,1,0,0,0.0,1.0


## Now let's start with splitting the data into train test sets and handle the outliers

- First we can notice by applying the logic on the tenure column in the X_train dataframe that there are now outliers in it.
- Now we notice too that there are no outliers in the Monthly charges column
- And now we can see too that we don't have any outliers in the total charges colum
- Now we may say that the data is ready for training.

In [79]:
from sklearn.model_selection import train_test_split

In [80]:
# First, let's split the data
# Now let's Define out X and y dataframes that are going to be used in the split
X = df.drop('Churn', axis=1)
y = df['Churn']

# Now let's start splitting the data to different test/train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (5634, 26)
Shape of X_test: (1409, 26)
Shape of y_train: (5634,)
Shape of y_test: (1409,)


In [81]:
# Ok, now let's handle the outliers in tenure, monthly charges, and total, charges column for the X_train data set
X_train.head(10)

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,PaperlessBilling,MonthlyCharges,TotalCharges,DSL_connection,Fiber_connection,has_internet,montly_contract,annual_contract,two_years_contract,electronic_check_payment,mailed_check_payment,bank_transfer_payment,credit_card_payment,streamsTV,gender_Female,gender_Male
2142,0,0,1,21,1,0,1,0,1,0,0,64.85,1336.80,1,0,1,0,1,0,0,1,0,0,1,1.0,0.0
1623,0,0,0,54,1,1,0,1,0,0,1,97.20,5129.45,0,1,1,0,0,1,0,0,1,0,1,1.0,0.0
6074,0,1,0,1,0,0,0,0,0,0,1,23.45,23.45,1,0,1,1,0,0,1,0,0,0,0,0.0,1.0
1362,0,0,0,4,1,0,0,0,0,0,1,70.20,237.95,0,1,1,1,0,0,1,0,0,0,0,0.0,1.0
6754,0,0,1,0,1,1,1,1,0,1,1,61.90,0.00,1,0,1,0,0,1,0,0,1,0,0,0.0,1.0
1212,0,0,0,7,1,0,0,0,0,0,1,69.55,521.35,0,1,1,1,0,0,1,0,0,0,0,0.0,1.0
2722,0,1,1,32,1,0,0,0,0,0,0,18.95,613.95,0,0,0,0,0,1,0,1,0,0,0,0.0,1.0
4006,0,1,1,72,1,0,0,0,0,0,1,19.80,1414.65,0,0,0,0,0,1,0,0,0,1,0,0.0,1.0
6791,0,0,0,19,0,0,0,0,1,0,1,39.65,733.35,1,0,1,1,0,0,1,0,0,0,1,0.0,1.0
5466,0,0,1,10,1,0,0,0,0,0,1,19.95,219.50,0,0,0,0,1,0,0,0,1,0,0,0.0,1.0


In [87]:
# Now we will take a look at the different outliers in the training set
# Starting by the tenure column
Q1_tenure = X_train['tenure'].quantile(0.25)
Q3_tenure = X_train['tenure'].quantile(0.75)
IQR_tenure = Q3_tenure - Q1_tenure
outliers = X_train[(X_train['tenure'] < (Q1_tenure - 1.5 * IQR_tenure)) | (X_train['tenure'] > (Q3_tenure + 1.5 * IQR_tenure))]
print(outliers)

Empty DataFrame
Columns: [SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, PaperlessBilling, MonthlyCharges, TotalCharges, DSL_connection, Fiber_connection, has_internet, montly_contract, annual_contract, two_years_contract, electronic_check_payment, mailed_check_payment, bank_transfer_payment, credit_card_payment, streamsTV, gender_Female, gender_Male]
Index: []


In [84]:
# Now let's take a look at the Monthly_charges column
Q1_monthly = X_train['MonthlyCharges'].quantile(0.25)
Q3_monthly = X_train['MonthlyCharges'].quantile(0.75)
IQR_monthly = Q3_monthly - Q1_monthly
outliers_monthly = X_train[(X_train['MonthlyCharges'] < (Q1_monthly - 1.5 * IQR_monthly)) | (X_train['MonthlyCharges'] > (Q3_monthly + 1.5 * IQR_monthly))]
print(outliers_monthly)

Empty DataFrame
Columns: [SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, PaperlessBilling, MonthlyCharges, TotalCharges, DSL_connection, Fiber_connection, has_internet, montly_contract, annual_contract, two_years_contract, electronic_check_payment, mailed_check_payment, bank_transfer_payment, credit_card_payment, streamsTV, gender_Female, gender_Male]
Index: []


In [85]:
# Now let's take a look at the outliers in the total charges column.
Q1_total = X_train['TotalCharges'].quantile(0.25)
Q3_total = X_train['TotalCharges'].quantile(0.75)
IQR_total = Q3_total - Q1_total
outliers_total = X_train[(X_train['TotalCharges'] < (Q1_total - 1.5 * IQR_total)) | (X_train['TotalCharges'] > (Q3_total + 1.5 * IQR_total))]
print(outliers_total)

Empty DataFrame
Columns: [SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, PaperlessBilling, MonthlyCharges, TotalCharges, DSL_connection, Fiber_connection, has_internet, montly_contract, annual_contract, two_years_contract, electronic_check_payment, mailed_check_payment, bank_transfer_payment, credit_card_payment, streamsTV, gender_Female, gender_Male]
Index: []
